<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/6_Transformers/1_Transformers.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Transformers: Solo Necesitamos Prestar Atención

En la clase anterior, analizamos los mecanismos de atención, un método omnipresente en los modelos modernos de Deep Learning. La atención es un concepto que ayudó a mejorar el rendimiento de las aplicaciones de traducción automática neuronal. En esta clase, veremos `The Transformer`, un modelo que utiliza **sólo** mecanismos de atención para aumentar la velocidad con la que se pueden entrenar estos modelos. 

El Transformer fue propuesto en el artículo ["Attention is All you Need"](https://arxiv.org/abs/1706.03762). En esta publicación, intentaremos simplificar un poco las cosas e introducir los conceptos uno por uno para que sea más fácil de entender para las personas sin un conocimiento profundo del tema.

## Estructura General

Comencemos mirando el modelo como una sola caja negra. En una aplicación de traducción automática, tomaría una oración en un idioma y generaría su traducción en otro.

![Imgur](https://i.imgur.com/EQicdb3.png)

Al abrir a Optimus Prime, vemos un encoder, un decoder y conexiones entre ellos.

![Imgur](https://i.imgur.com/xqPqBL1.png)

El encoder es en realidad una pila de encoders (el paper apila seis de ellos uno encima del otro; no hay nada mágico en el número seis, definitivamente se puede experimentar con otros arreglos). El decoder es también una pila del mismo número de decoders.

![Imgur](https://i.imgur.com/qejWps3.png)

Los encoders son todos idénticos en estructura (**pero no comparten pesos**). Cada uno se divide en dos subcapas:

![Imgur](https://i.imgur.com/9rYSc56.png)

Las entradas del encoder fluyen primero a través de una capa de **Auto-Atención (Self-Attention)**, una capa que ayuda al codificador a ver otras palabras en la oración de entrada mientras codifica una palabra específica. Veremos más de cerca la Auto-Atención más adelante en el notebook.

Las salidas de la capa de Auto-Atención se envían a una red feed-forward (otro nombre que reciben los MLP). La misma red de feed-forward se aplica de forma independiente a cada posición.

El decoder tiene ambas capas, pero entre ellas hay una capa de atención común que ayuda al decoder a enfocarse en partes relevantes de la oración de entrada.

![Imgur](https://i.imgur.com/obppn2C.png)

## Flujo de Tensores



Ahora que hemos visto los componentes principales del modelo, comencemos a ver los diversos tensores y cómo fluyen entre estos componentes para convertir la entrada de un modelo entrenado en una salida.

Como es el caso en las aplicaciones NLP en general, comenzamos convirtiendo cada palabra de entrada en un vector utilizando un algoritmo de embedding.

![Imgur](https://i.imgur.com/5pF2VfG.png)

La transformación a embeddings solo ocurre en el encoder de más abajo. La abstracción que es común a todos los encoders es que reciben una lista de vectores, cada uno de tamaño 512: en el encoder inferior, serían los embeddings de las palabras, pero en los demás, sería la salida del encoder que está directamente debajo. El tamaño de esta lista es un hiperparámetro que podemos configurar; básicamente, sería la longitud de la oración más larga en nuestro conjunto de datos de entrenamiento.

Después de generar los embeddings de las palabras en nuestra secuencia de entrada, cada una de ellas fluye a través de cada una de las dos capas del encoder.

![Imgur](https://i.imgur.com/EPxkIRI.png)

Aquí comenzamos a ver una propiedad clave del Transformer, que es que la palabra en cada posición fluye a través de su propio camino en el encoder. Existen dependencias entre estas rutas en la capa de autoatención. Sin embargo, la capa feed-forward no tiene esas dependencias y, por lo tanto, las diversas rutas se pueden ejecutar en paralelo mientras fluye a través de esta capa.

![Imgur](https://i.imgur.com/UzTM32F.png)




## Auto-Atención

En la clase anterior, habíamos visto cómo mejoraba la performance de los modelos de secuencia a secuencia cuando el decoder prestaba atención a la frase entera procesada por el encoder antes de predecir cuál sería el siguiente token. Recordemos un poco ese proceso.

1. El decoder miraba el conjunto de estados ocultos que recibió del encoder (cada estado oculto está más asociado con una determinada palabra en la oración de entrada)
2. Se asignaba una puntuación a cada estado oculto (calculada por alguna función de atención como aditiva, producto escalar o producto escalar generalizado) y luego se les pasaba esos valores a un softmax
3. Se multiplicaba cada estado oculto por su puntaje "softmaxeado", amplificando así los estados ocultos con puntajes altos y diluyendo los estados ocultos con puntajes bajos
4. Se sumaba todo y obteníamos el vector de contexto

![Imgur](https://i.imgur.com/0CnIjmA.png)





En este proceso, a la hora de calcular esos puntajes tomábamos el estado oculto actual del decoder como query, y los estados ocultos de cada tiempo del encoder eran tanto las keys como los values.

![Imgur](https://i.imgur.com/O9ocqXJ.png)

Ahora, al hablar de auto-atención, las queries, las keys y los values provienen del mismo lugar: **los tokens de entrada**. A medida que el modelo procesa cada palabra (cada posición en la secuencia de entrada), la auto-atención le permite observar otras posiciones en la secuencia de entrada en busca de pistas que puedan ayudar a codificar mejor esta palabra.

Primero veamos cómo calcular la autoatención usando vectores, luego procedamos a ver cómo se implementa realmente, usando matrices.

El primer paso para calcular la autoatención es crear tres vectores a partir de cada uno de los vectores de entrada del encoder (en este caso, el embedding de cada palabra). Entonces, para cada palabra, creamos un vector query, un vector key y un vector value. Estos vectores se crean multiplicando el embedding por tres matrices que entrenamos durante el proceso de entrenamiento.

Tenga en cuenta que estos nuevos vectores son más pequeños en dimensión que el vector embedding. Su dimensionalidad es 64, mientras que los embeddings y los vectores de entrada/salida de los encoders tienen una dimensionalidad de 512. No es obligatorio que sean más pequeños, esta es una elección de arquitectura que explicaremos más adelante.

![Imgur](https://i.imgur.com/tXhnp1U.png)

El segundo paso para calcular la autoatención es calcular una puntuación. Digamos que estamos calculando la autoatención de la primera palabra en este ejemplo, "Thinking". Necesitamos puntuar cada palabra de la oración de entrada contra esta palabra. La puntuación determina cuánta atención prestar en otras partes de la oración de entrada a medida que codificamos una palabra en una posición determinada.

La puntuación se calcula tomando el producto escalar del vector query con el vector key de la palabra respectiva que estamos puntuando. Entonces, si estamos procesando la autoatención de la palabra en la posición n. ° 1, el primer puntaje sería el producto escalar de q1 y k1. La segunda puntuación sería el producto escalar de q1 y k2.

![Imgur](https://i.imgur.com/LK5HwRT.png)

Los pasos tercero y cuarto consisten en dividir las puntuaciones por 8 (la raíz cuadrada de la dimensión de los vectores key utilizados en el paper: 64). Esto lleva a tener gradientes más estables. Podría haber otros valores posibles aquí, pero este es el predeterminado), luego pase el resultado a través de una operación softmax. Softmax normaliza las puntuaciones para que se comporten como una distribución de probabilidad (todas sean positivas y sumen 1).

![Imgur](https://i.imgur.com/EohRASF.png)

Esta puntuación softmax determina cuánto se enfocará cada palabra en esta posición. Claramente, la palabra en esta posición tendrá la puntuación de softmax más alta, pero a veces es útil prestar atención a otra palabra que sea relevante para la palabra actual.

El quinto paso es multiplicar cada vector de valor por la puntuación softmax (en preparación para sumarlos). La intuición aquí es mantener intactos los valores de las palabras en las que queremos centrarnos y diluir las palabras irrelevantes (multiplicándolas por números pequeños como 0,001, por ejemplo).

El sexto paso es sumar los vectores de valores ponderados. Esto produce la salida de la capa de autoatención en esta posición (para la primera palabra).

![Imgur](https://i.imgur.com/7ghvKCr.png)

## Múltiples Cabezales

El paper refina aún más la capa de autoatención al agregar un mecanismo llamado atención de "múltiples cabezales". Esto mejora el rendimiento de la capa de atención de dos maneras:

* Expande la capacidad del modelo para enfocarse en diferentes posiciones. Sí, en el ejemplo anterior, z1 contiene un poco de las otras posiciones, pero podría estar dominada por la propia palabra. Si estamos traduciendo una oración como “El animal no cruzó la calle porque estaba demasiado cansado”, sería útil saber si el "estaba" se refiere al animal o a la calle .

* Da a la capa de atención múltiples “subespacios de representación”. Como veremos a continuación, con la atención de varios cabezales no solo tenemos uno, sino varios conjuntos de matrices de pesos de Query/Key/Value (el Transformer utiliza ocho cabezales de atención, por lo que terminamos con ocho conjuntos para cada encoder/decoder) . Cada uno de estos conjuntos se inicializa aleatoriamente. Luego, después del entrenamiento, cada conjunto se usa para proyectar los embeddings de entrada (o los vectores de salida de los encoders/decoders inferiores) en un subespacio de representación diferente.

![Imgur](https://i.imgur.com/AspKg0z.png)

Si hacemos el mismo cálculo de autoatención que describimos anteriormente, solo ocho veces diferentes con diferentes matrices de peso, terminamos con ocho matrices Z diferentes.

![Imgur](https://i.imgur.com/i2dtHH6.png)

Esto nos deja con un pequeño desafío. La capa feed-forward no espera ocho matrices, espera una sola matriz (un vector para cada palabra). Así que necesitamos una forma de condensar estos ocho en una sola matriz.

¿Como hacemos eso? Concatenamos las matrices y luego las multiplicamos por una matriz de pesos adicional $W^O$.

![Imgur](https://i.imgur.com/rAgPw9N.png)



Y eso es todo el proceso de Auto-Atención con Múltiples Cabezales. Pongamos todo en un mismo gráfico para ver el pipeline completo.

![Imgur](https://i.imgur.com/2JljHSG.png)

## Codificación Posicional

Una cosa que falta en el modelo tal como lo hemos descrito hasta ahora es una forma de codificar el orden de las palabras en la secuencia de entrada. Debido a que ya no tenemos RNNs que procesen el problema de manera secuencial, esto puede llegar a ser un problema.

Para abordar esto, el Transformer agrega un vector a cada embedding de entrada. Estos vectores siguen un patrón específico que aprende el modelo, lo que le ayuda a determinar la posición de cada palabra o la distancia entre diferentes palabras en la secuencia. La intuición aquí es que agregar estos valores a los embeddings proporciona distancias significativas entre los vectores de embeddings una vez que se proyectan en vectores Q/K/V y durante la atención de producto escalar.

![Imgur](https://i.imgur.com/thGgU8N.png)

Hay muchas opciones de codificaciones posicionales, tanto aprendidas como fijas.

En este trabajo utilizaron funciones seno y coseno de diferentes frecuencias:
$$PE(pos,2i)=sin(pos/10000^{2i/embedsize})$$

$$PE(pos,2i+1)=cos(pos/10000^{2i/embedsize})$$

donde pos es la posición e i es la dimensión. Es decir, cada dimensión de la codificación posicional corresponde a una sinusoide. Las longitudes de onda forman una progresión geométrica de 2π a 10000⋅2π. Los autores eligieron esta función porque intuían que permitiría que el modelo aprendiera fácilmente a atender por posiciones relativas, ya que para cualquier desplazamiento k fijo, $PE(pos+k)$ puede representarse como una función lineal de $PE(pos)$

La siguiente imagen muestra cómo se ve este patron.
![Imgur](https://i.imgur.com/PGQFIKU.png)



## Conexiones Residuales




Un detalle de la arquitectura del encoder que debemos mencionar antes de continuar es que cada subcapa (autoatención, feed-forward) en cada encoder tiene una conexión residual a su alrededor, y va seguida de un paso de normalización de capa.

![Imgur](https://i.imgur.com/3KZCxga.png)

La normalización es buena para los modelos. Reduce el tiempo de entrenamiento, evita que el modelo se sesgue hacia características de mayor valor y no permite que los pesos exploten por todos lados restringiéndolos a un cierto rango. Por todo esto, no es deseable entrenar un modelo con descenso de gradiente con características no normalizadas.

En la normalización de capa, tomamos la media y la desviación estandar de todas las características de una sola oración y aplicamos la siguiente fórmula:

$$x_{norm} = \frac{x - mean(x)}{std(x)}$$

![Imgur](https://i.imgur.com/sbsytjy.png)

Esto también se aplica a las subcapas del decoder. Si tuviéramos que pensar en un transformer con 2 encoders y 2 decoders apilados, se vería así:

![Imgur](https://i.imgur.com/EI0DeiK.png)


## Etapa de Decodificación



Ahora que hemos cubierto la mayoría de los conceptos en el lado del encoder, básicamente también sabemos cómo funcionan los componentes de los decoders. Pero echemos un vistazo a cómo funcionan juntos.

El encoder comienza procesando la secuencia de entrada. La salida del encoder superior se transforma luego en un conjunto de vectores de atención K y V. Estos deben ser utilizados por cada decoder en su capa de "atención encoder-decoder" que ayuda al decoder a enfocarse en los lugares apropiados en la secuencia de entrada:

![](https://i.imgur.com/Ymqnciw.gif)

Los siguientes pasos repiten el proceso hasta que se alcanza un símbolo especial que indica que el decoder del transformer ha completado su salida. La salida de cada paso se alimenta al decoder inferior en el siguiente paso de tiempo, y los decoders van pasando hacia arriba sus resultados de decodificación al igual que lo hicieron los encoders. Y tal como hicimos con las entradas del encoder, generamos el embedding y agregamos codificación posicional a esas entradas del decoder para indicar la posición de cada palabra.

![](https://i.imgur.com/FHUcoiw.gif)

Las capas de autoatención en el decoder funcionan de una manera ligeramente diferente a la del encoder:

En el decoder, la capa de autoatención solo puede atender posiciones **anteriores** en la secuencia de salida. Esto se hace **enmascarando** posiciones futuras (configurándolas en -inf) antes del paso softmax en el cálculo de autoatención.

La capa de "Atención de encoder-decoder" funciona como la autoatención de varios cabezales que explicamos antes, excepto que crea su matriz de queries a partir de la capa debajo de ella y toma la matriz de keys y values de la salida de la pila del encoder.

## Capa Densa de Predicción

La pila del decoder genera un vector de punto flotante. ¿Cómo convertimos eso en una palabra? Ese es el trabajo de la capa densa final, seguida de una capa Softmax. Esta capa transforma el vector producido por la pila de decoders en un vector mucho, mucho más grande llamado vector logits.

Supongamos que nuestro modelo conoce 10 000 palabras únicas en inglés (el "vocabulario de salida" de nuestro modelo) que aprendió de su conjunto de datos de entrenamiento. Esto haría que el vector logits tuviera 10 000 celdas de ancho, cada celda correspondiente a la puntuación de una palabra única. 

La capa softmax luego convierte esos puntajes en probabilidades (todos positivos, todos suman 1.0). Se elige la celda con la probabilidad más alta y la palabra asociada a ella se produce como salida para este paso de tiempo.

![Imgur](https://i.imgur.com/eElpr7z.png)